# 使用GPU

https://www.bilibili.com/video/BV1z5411c7C1?p=1

先看下有没有GPU

In [1]:
!nvidia-smi

Wed Dec  1 08:47:36 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.59       Driver Version: 462.59       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 165... WDDM  | 00000000:02:00.0 Off |                  N/A |
| N/A   40C    P8     2W /  N/A |    194MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 计算设备

所有的框架，默认都是在cpu上计算的，所以如果想使用gpu，必须提前先指定一下。

In [5]:
import torch
from torch import nn

# 指定使用cpu作为计算设备（也是默认选项）
print(torch.device('cpu'))

# 这里torch使用cuda代替gpu，但是cuda和gpu不是那么的等价，
# 所以表达上其实存在点问题，但是没什么大的影响，就继续沿用了
print(torch.cuda.device('cuda')) # 没有指明数字，默认就是第0个gpu


# 多个gpu的话，指定数字，如第1个gpu
print(torch.cuda.device('cuda:1'))

cpu


如果要指定多个GPU的话，比如单机多卡训练，这时候，写法就不太一样，下面这种是错的    
```python   
print(torch.cuda.device('cuda:[0,1]'))
``` 
参考：[pytorch指定用多张显卡训练_pytorch多gpu并行训练](https://blog.csdn.net/weixin_42291186/article/details/112207044)

查询可用gpu的数量

In [8]:
torch.cuda.device_count()

1

这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [10]:
torch.device("cuda:0")

device(type='cuda', index=0)

In [12]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count()>=1:
        return torch.device(f"cuda:{i}")
    return torch.device("cpu")

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    gpu_num=torch.cuda.device_count()
    devices=[torch.device(f"cuda:{i}") for i in range(gpu_num)]
    return devices if devices else [torch.device('cpu')]

try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cuda', index=10),
 [device(type='cuda', index=0)])

这里显示的内容和李沐老师的不一样
+ windows下，其实我的gpu只有1个，index应该只会为0
+ 但是在`try_gpu(10)`中，还是返回`index=10`，说明程序里设备序号和实际的物理设备序号有一层映射，不同的框架下，这个映射规定有所不同，不要太纠结。用的时候注意点就行

## 张量所在的设备

In [13]:
x=torch.tensor([1,2,3])
x.device

device(type='cpu')

+ 可以使用`device`查询当前张量所在的设备，默认就是在cpu
+ 但是可以在创建的时候指定，放到gpu上

In [14]:
X=torch.ones(2,3,device=try_gpu())
X.device

device(type='cuda', index=0)

这个的执行速度明显要比在cpu上慢一点

In [15]:
B=torch.ones(2,3,device=torch.device("cuda:0"))
B.device
# 也可以直接指定设备，没必要写那个函数

device(type='cuda', index=0)

这里有一个问题需要注意：
+ 默认张量是创建在cpu上，所以两个张量计算，也是在cpu上进行
+ 如果张量创建在了gpu上，那么张量计算也是发生在gpu上，但是需要保证，两个张量位于一块GPU。。有多卡的人需要注意这一点
+ 假设X是在第一张GPU卡上创建的张量，Y是第二张GPU，那么如果需要计算，需要先将其中一个张量移动到另一个张量所在的卡上

```python
Z=X.cuda(1)
# 表示将X的值从一个gpu0复制到gpu1
```

+ 实际上，张量放在不同的GPU上，也可以移动之后再去计算。
+ 但是对于CPU和GPU混用的机器来说，由于数据在CPU和GPU之间交换速度很慢，所以考虑到这点，一旦某个网络有一部分在CPU，有一部分在GPU，就直接给你报个错，因为这样很难保证性能
+ 应该是底层实现不容易，同时利用cpu和gpu的资源很难实现的很好。

## 神经网络和GPU

In [16]:
net=nn.Sequential(nn.Linear(3,1))
net=net.to(device=try_gpu()) # Moves and/or casts the parameters and buffers.

net(X)

tensor([[0.9463],
        [0.9463]], device='cuda:0', grad_fn=<AddmmBackward>)

这里注意一个区别：
+ torch.device("cuda:1")是直接指定设备
+ net.to()这个函数是将参数等移动到/映射到另一个地方

确认模型参数存储在同一个GPU上

In [17]:
net[0].weight.data.device

device(type='cuda', index=0)